In [1]:
#path setup
import os
import sys

project_root_path = "/Users/cmazzoleni/Documents/GitHub/CuboidAbstractionViaSeg"
# Add the src directory to sys.path
sys.path.append(os.path.join(project_root_path))
import numpy as np
import open3d as o3d
import argparse
from src.utils.dataset_utils import create_directory
from src.visualizers.Open3dVisualizer import Open3DVisualizer
from src.utils.json_utils import load_cuboid_data
from src.utils.plotting_utils import load_mesh_data, load_pointcloud_data
import pprint
import cv2
from PIL import Image

# Help functions


# Constant values


In [2]:
extrinsic_front = np.array([
        [1, 0, 0, 0],    
        [0, 1, 0, 0],    
        [0, 0, 1, 1],  
        [0, 0, 0, 1]])


flip_transform = np.array([[1, 0, 0, 0], 
                           [0, -1, 0, 0], 
                           [0, 0, 1, 0],  
                           [0, 0, 0, 1]])

# Prediction PCD


In [3]:
prediction_depth_image_path = "try_decode.png"
prediction_depth_image = cv2.imread(prediction_depth_image_path, cv2.IMREAD_UNCHANGED)  # Read the image
prediction_depth_image = prediction_depth_image.astype(np.uint8)

min_depth = np.min(prediction_depth_image)
max_depth = np.max(prediction_depth_image)

print (min_depth)
print (max_depth)

#prediction_depth_image = prediction_depth_image / 65535.0
prediction_depth_image = np.where((prediction_depth_image == 0) | (prediction_depth_image== 1), np.nan, prediction_depth_image)


44
255


In [4]:
prediction_depth_image_path = "try_decode.png"
prediction_depth_image = Image.open(prediction_depth_image_path)
prediction_depth_image = np.array(prediction_depth_image)
prediction_depth_image = prediction_depth_image.astype(np.uint16)
#prediction_depth_image = prediction_depth_image / 65535.0

# GT PCD

In [16]:

gt_depth_image_path = "gt_chair.png"
gt_depth_image = cv2.imread(gt_depth_image_path, cv2.IMREAD_UNCHANGED)  # Read the image
gt_depth_image = gt_depth_image.astype(np.float32)

"""
gt_depth_image_path = "depth_image.png"

gt_depth_image = o3d.io.read_image(gt_depth_image_path)
gt_depth_image = np.asarray(gt_depth_image)
min_depth = gt_depth_image.min()
max_depth = gt_depth_image.max()
print(min_depth)
print(max_depth)
#create a figure with the same dimesnions as the image
gt_depth_image = Image.fromarray((gt_depth_image * 255).astype(np.float32))

"""
min_depth = np.min(gt_depth_image)
max_depth = np.max(gt_depth_image)
print(min_depth)
print(max_depth)

gt_depth_image = (gt_depth_image - min_depth) / (max_depth - min_depth)
#gt_depth_image = np.where((gt_depth_image == 0) | (gt_depth_image== 1), np.nan, gt_depth_image)
#min_depth = np.min(gt_depth_image)
#max_depth = np.max(gt_depth_image)
#print(min_depth)
#print(max_depth)


0.0
255.0


In [8]:
gt_depth_image_path = "gt_chair.png"
gt_depth_image = Image.open(gt_depth_image_path)
gt_depth_image = np.array(gt_depth_image)
gt_depth_image = gt_depth_image.astype(np.uint16)

# Mesh 

In [6]:
mesh_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/raw/03001627/185bcb9bcec174c9492d9da2668ec34c/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()


mesh.translate(-mesh.get_center())


bounding_box = mesh.get_axis_aligned_bounding_box()


max_extent = np.max(bounding_box.get_extent())


scale_factor = 1.0 / max_extent
mesh.scale(scale_factor, center=mesh.get_center())



TriangleMesh with 3356 points and 2720 triangles.

# Visualization

In [9]:
visualizer = Open3DVisualizer()
visualizer.add_geometry(mesh)

#assert gt_depth_image.shape[0]==prediction_depth_image.shape[0]
#assert gt_depth_image.shape[1]==prediction_depth_image.shape[1]

intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(width=prediction_depth_image.shape[1], 
                         height=prediction_depth_image.shape[0], 
                         fx=692.8203230275511,  # Focal length in x
                         fy=692.8203230275511,  # Focal length in y
                         cx=prediction_depth_image.shape[1] / 2.0,  # Principal point in x
                         cy=prediction_depth_image.shape[0] / 2.0)  # Principal point in y


prediction_depth_image_o3d = o3d.geometry.Image(prediction_depth_image)
gt_depth_image_o3d = o3d.geometry.Image(gt_depth_image)
  
# Create a point cloud from the depth map using both intrinsic and extrinsic parameters
pcd_prediction = o3d.geometry.PointCloud.create_from_depth_image(
    prediction_depth_image_o3d, 
    intrinsic,
    extrinsic= extrinsic_front
)



pcd_gt = o3d.geometry.PointCloud.create_from_depth_image(
    gt_depth_image_o3d, 
    intrinsic,
    extrinsic= extrinsic_front
)
pcd_gt.paint_uniform_color([255,0,0])


# Transform the point cloud
pcd_prediction.transform(flip_transform)
pcd_gt.transform(flip_transform)

o3d.visualization.draw_geometries([ pcd_prediction, mesh, pcd_gt])

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


: 